# Create Event settings

Here we create a toml file specifying the settings for a given event.

In [ ]:
from pathlib import Path
from hydromt_sfincs import SfincsModel
from pprint import pprint
from os import makedirs
import pandas as pd
import tomli_w

# import sys
# sys.path.insert(0, 'c:/Repositories/FloodAdapt/')

from flood_adapt.object_model.site import Site
from flood_adapt.object_model.hazard.event.eventset import EventSet

## Base FloodAdapt path

In [ ]:
model_root = Path('c:/Repositories/DT-flood/FloodAdapt_database/Humber/')

As a first check, load in site.toml of the relevant model/database

In [ ]:
site_fn = model_root / 'static' / 'site' / 'site.toml'
site_obj = Site.load_file(site_fn)

pprint(vars(site_obj.attrs))

In [ ]:
site_obj.attrs.sfincs.overland_model

## Read in overland SFINCS model

In [ ]:
sf_root = model_root / 'static' / 'templates' / site_obj.attrs.sfincs.overland_model
print(sf_root)
sf = SfincsModel(root=sf_root, mode='r')
sf.read()

## Create empty event

We will create an empty event, where we only use the baseline/default forcings we specified when building the SFINCS overland model and add nothing else. This most convienently done in the 'Historical_nearshore' event. This requires a tides timeseries, so we take the bzs forcing created during the SFINCS setup and write it away as a timeseries in the correct event folder.

The 'synthetic' template requires tides setup through the site.toml with tide.source='harmonic'. Using 'Historical_offshore' template will override the SFINCS meteo forcings by downloading US(!!) meteo data and set those up as forcing.



In [ ]:
# Create folder for empty event
event_name = 'Empty_Event'
event_path = model_root / 'input' / 'events' / event_name
if not event_path.exists():
    makedirs(event_path)


In [ ]:
## Prep sfincs bzs as timeseries
pd_bzs = pd.DataFrame(data=sf.forcing['bzs'].values.transpose(), index=sf.forcing['bzs'].time.values)
pd_bzs.to_csv((event_path / 'bzs_timeseries.csv'),header=None)

Below we specify all the entries into the .toml file for the event:
- `event_mode`: Risk type of the event. One of 'single_event' or 'risk'. The latter makes use of the site.risk settings
- `event_template`: Base template of the event. One of 'synthetic', 'Historical_nearshore', 'Historical_offshore', 'Historical_hurricane'
- `event_timing`: One of 'historical' or 'idealized'
- `event_time`: dates of the event, can also be specified as a time before and after a reference time (usually when using forcings through shape sources?). Here taken from the SFINCS config
- `event_tide`: Waterlevel forcings used in the event (a dict containing settings):
    - `source`: Where to take tides from, one of 'harmonic', 'timeseries', 'model'
    - `harmonic_amplitude`: Tide amplitude (UnitfulLength, used when source='harmonic')
    - `timeseries_file`: Path to file containing timeseries (used when source='timeseries')
- `event_wind`: Wind forcings used in the event (dict containing settings):
    - `source`: Where to take winds from, one of 'track', 'map', 'constant', 'timeseries', 'shape', 'none'
    - `constant_speed`: wind speed (used when source='constant', UnitfulVelocity)
    - `constant_direction`: wind direction (used when source='constant', UnitfulDirection)
- `event_rain`: Precipitation forcing used in event (dict containing settings):
    - `source`: Where to take precipitation from, one of 'track', 'map', 'constant', 'timeseries', 'shape', 'none'
    - `increase`: projected relative increase in rainfall (optional)
    - `constant_intensity`: Amount of precipitation (used when source='constant', UnitfulIntensity)
    - `timeseries_file`: path to file containing timeseries (used when source='timeseries')
    - `shape_type`: Intensity profile of precipitation event, one of 'gaussian', 'block', 'triangle', 'scs' (a file in /static/scs) (used when source='shape')
    - `cumulative`: Total amount of rainfall in event (used when source='shape', UnitfulLength)
    - `shape_duration`: Time interval of rainfall event (used when source='shape')
    - `shape_peak_time`: When peak rainfall occurs (used when source='shape')
    - `shape_start_time`: When rainfall event begins (used when source='shape')
    - `shape_end_time`: When rainfall event ends (used when source='shape')
- `event_surge`: Additional water level surge on top of tides (dict containing settings):
    - `source`: Where to take surge data from, one of 'shape', 'none'
    - `shape_type`: surge profile (string, default 'gaussian', not same type of option as event_rain.shape_type)
    - `shape_duration`: Time interval of surge
    - `shape_peak_time`: When peak of the surge occurs
    - `shape_peak`: Height of surge peak
- `event_river`: River discharges used in event (list of dicts, same length as site.river)
    - `source`: Where to take discharges from, one of 'constant', 'timeseries', 'shape'
    - `constant_discharge`: River discharge (list of UnitfulDischarge, used when source='constant')
    - `timeseries_file`: path to file containing discharge timeseries (list of files, used when source='timeseries')
    - `shape_type`: Shape of discharge profile (list, see event_rain.shape_type for options, used when source='shape')
    - `cumulative`: Total amount of discharge during event (list, used when source='shape')
    - `shape_duration`: Time interval of discharge event (list, used when source='shape')
    - `shape_peak_time`: When peak discharge occurs (list, used when source='shape')
    - `shape_start_time`: When discharge event begins (list, used when source='shape')
    - `shape_end_time`: When discharge event ends (list, used when source='shape')
- `event_water_offset`: Offset of waterlevels vs model base on top of site.slr settings (UnitfulLength)


In [ ]:

event_mode = 'single_event'
event_template = 'Historical_nearshore'
event_timing = 'historical'
event_time = {'start_time': sf.config['tstart'].strftime('%Y%m%d %H%M%S'),
              'end_time': sf.config['tstop'].strftime('%Y%m%d %H%M%S')}
event_tide = {'source': 'timeseries',
              'timeseries_file': 'bzs_timeseries.csv'}
event_wind = {'source': 'none'}
event_rain = {'source': 'none'}
event_surge = {'source': 'none'}
event_river = {'source': 'timeseries'}
event_water_offset = {'value': 0,
                      'units': 'meters'}

## Create event dict
Put all the above specified settings in a dict with the necessary keys

In [ ]:
event_dict = {
    'name': event_name,
    'mode': event_mode,
    'template': event_template,
    'timing': event_timing,
    'time': event_time,
    'tide': event_tide,
    'wind': event_wind,
    'rainfall': event_rain,
    'river': [event_river],
    'surge': event_surge,
    'water_level_offset': event_water_offset
}

## Save event toml

In [ ]:
event_fn = event_path / f"{event_name}.toml"
with open(event_fn,'wb') as f:
    tomli_w.dump(event_dict,f)

## Test load

In [ ]:
loaded_event = EventSet.load_file(event_fn)

In [ ]:
loaded_event.attrs